In [4]:
# v tomto kode nacitame udaje o anatomickych zlozkach Hetionetu a konvertujeme ich 
# do RDF grafu "https://anatomy_ontology.org"

# importujeme potrebne kniznice
import re
from rdflib import Namespace, Literal, URIRef, Dataset
from rdflib.namespace import RDF, RDFS, SKOS, OWL, DC

# cesta k obo suboru UBERON
obo_file = '/Users/simonsnopko/Desktop/Škola/bakalarska praca/uberon-simple.obo'

# definujeme namespace pre uberon a bioregistry
UBERON = Namespace("http://purl.obolibrary.org/obo/")
BIOREGISTRY = Namespace("https://bioregistry.io/")

# vytvorime dataset, meno grafu a do premennej new_g priradime menovity graf
ds = Dataset()
graph_name = URIRef("https://anatomy_ontology.org")
new_g = ds.graph(identifier=graph_name)

# nabindujeme namespaces ku grafu
new_g.bind("bioregistry", BIOREGISTRY)
new_g.bind("skos", SKOS)

# otvorime obo subor a precitame jeho obsah
with open(obo_file, 'r') as f:
    content = f.read()

# rozdelime obsah podla termov
terms = content.split("[Term]")

# prechadzame cez vsetky termy a odstranujeme prazdne medzery, tabulatory, nove riadky 
# na zaciatku a na konci retazca
for term in terms:
    term = term.strip()
    if not term:
        continue

    # vytvorime prazdny slovnik
    term_dict = {}
    # kazdy term sa rozdeli na jednotlive riadky
    for line in term.split('\n'):
        # kazdy riadok sa podľa dvojbodky a jednej alebo viacerých medzier rozdeli (: ) pomocou funkcie split 
        key_value = re.split(r':\s+', line, maxsplit=1)
        # kontroluje sa ci rozdelenie vratilo dve casti
        if len(key_value) == 2:
            # do premennej sa ulozi key a value
            key, value = key_value
            # key a value sa pridaju do slovníka term_dict
            term_dict[key] = value

    # zo slovnika term_dict ziskame udaje o termoch pomocou nazvov klucov
    id_ = term_dict.get('id')
    name = term_dict.get('name')
    definition = term_dict.get('def')
    subset = term_dict.get('subset')

    # ak existuju vsetky udaje
    if id_ and name and definition and subset:
        # vytvorime URI pre bioregistry
        bioregistry_uri = URIRef(BIOREGISTRY + id_.replace('_', ':'))
        # pridame triplety do grafu
        new_g.add((bioregistry_uri, SKOS.prefLabel, Literal(name)))
        new_g.add((bioregistry_uri, SKOS.definition, Literal(definition)))
        new_g.add((bioregistry_uri, SKOS.narrower, Literal(subset)))
        new_g.add((bioregistry_uri, SKOS.notation, Literal(id_)))  

# ulozime graf do suboru v nquads formate
ds.serialize(destination="anatomy created ontology.nq", format="nquads")


<Graph identifier=N0d63cb0420734c2f908dab4e21a5416d (<class 'rdflib.graph.Dataset'>)>